# ECE/CS 434 | MP5: HMM
<br />
<nav>
    <span class="alert alert-block alert-warning">Due at 11:59PM Apr 15th 2024 on Gradescope</span>
</nav><br> 
 

## Objective
In this MP, you will:
- Explore two fundamental problems that are of interest to Hidden Markov Models (HMM):
    - What is the probability of a specific sequence of observations occuring given an HMM?
    - What is the optimal hidden state sequence given a sequence of observations and an HMM?
- Implement either the forward or backward algorithm to answer the first problem.
- Implement the Viterbi algorithm to answer the second problem.

---
## References
Material in this MP is adapted from the iconic paper *A Tutorial on Hidden Markov Models and Selected Applications in Speech Recognition* by Lawrence R. Rabiner. You are encouraged to follow the pseudocode in this paper for your implementation.

You are highly encouraged to watch [this video](https://www.youtube.com/watch?v=MPeedE6Odj0) by Prof.Patterson from Westmont College before beginning the MP.

---
## Imports & Setup

### Installing requirements correctly

First. we will make sure that the correct versions of required modules are installed. This ensures that your local Python environment is consistent with the one running on the Gradescope autograder. Just convert the following cell to code and run:

<div class="alert alert-block alert-info"><b>Note:</b> It's preferred that your local environment matches the autograder to prevent possible inconsistencies. However, if you're running into annoying Python version issues but haven't had any issues getting consistent results on the autograder, there is no need to stress over it. Just skip for now and come back when you do encounter inconsistencies:) Ditto below.
</div>

<div class="alert alert-block alert-info"><b>WARNING:</b> ENSURE THE FOLLOWING CELL IS MARKDOWN OR DELETED BEFORE SUBMITTING. THE AUTOGRADER WILL FAIL OTHERWISE.
</div>

if __name__ == '__main__':
    import sys
    !{sys.executable} -m pip install -r requirements.txt

### Your imports
Write your import statements below. If Gradescope reports an error and you believe it is due to an unsupported import, check with the TA to see if it could be added.

In [35]:
import numpy as np
import random
import scipy
import pandas as pd

def display_table(data):
    from IPython.display import HTML, display

    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>{}</h4><td>".format(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

---
## Constructing an HMM
Recall that an HMM consists of three parameters:
- $A$, the state transition probability matrix of size $N\times N$, where $N$ represents the number of hidden states(or true states).
- $B$, the observation probability matrix of size $N\times M$, where $M$ represents the number of observed states.
- $\pi$, an initial distribution vector of size $1\times N$

In this MP, we will be randomly generating these parameters. Run the cell below to see what a model looks like and how to access it. Notice you'll get slightly different numbers each time you run due to the random functions used.

In [36]:
if __name__ == '__main__':
    def generate_matrix(R, C, mode):
        model = np.zeros((R, C))
        for i in range(R):
            row = np.zeros(C)
            if(mode == 0):
                while(row.sum() != 1):
                    row = np.random.normal(0, 2, C)
                    row = row - row.min()
                    row = row / row.sum()
            elif(mode == 1):
                while(row.sum() != 1):
                    row = np.random.dirichlet(np.ones(C),size=1)
                    row = row / row.sum()
            elif(mode == 2):
                while(row.sum() != 1):
                    row = np.random.gamma(5, 5, C)
                    row = row / row.sum()
            model[i] = row
        return model

    example_model = {'A': generate_matrix(5, 5, 0),
                     'B': generate_matrix(5, 3, 0),
                     'pi': pi}
    print('A - State Transition Probability Matrix')
    print(pd.DataFrame(example_model['A']))
    print('The probability of transitioning from state 2 to state 3 is ' + str(example_model['A'][2][3]))
    print('\n\nB - Observation Probability Matrix')
    print(pd.DataFrame(example_model['B']))
    print('The probability of being in hidden state 1 while observing observable state 2 is ' + str(example_model['B'][1][2]))

A - State Transition Probability Matrix
          0         1         2         3         4
0  0.114590  0.000000  0.288758  0.459017  0.137636
1  0.315081  0.222022  0.308098  0.000000  0.154799
2  0.235511  0.000000  0.337005  0.123618  0.303866
3  0.000000  0.575138  0.114705  0.091806  0.218351
4  0.313785  0.313495  0.194560  0.178159  0.000000
The probability of transitioning from state 2 to state 3 is 0.12361835392744415


B - Observation Probability Matrix
          0    1         2
0  0.496030  0.0  0.503970
1  0.815616  0.0  0.184384
2  0.976593  0.0  0.023407
3  0.238118  0.0  0.761882
4  0.182572  0.0  0.817428
The probability of being in hidden state 1 while observing observable state 2 is 0.18438426054839976


---
## Problem 1. Determining the Best Model
You are given a sequence of observations, $O$, and a few candidate HMMs, $\{\lambda_1=(A_1, B_1, \pi_1), \lambda_2=(A_2, B_2, \pi_2)...\}$. Your task is to find the model that best describes the sequence of observations. 

You could approach this problem by calculating $P(O\mid\lambda)$ for every $\lambda$, and returning the index of the model yielding the highest likelihood.

<div class="alert alert-block alert-info"><b>Hint:</b> The Forward Algorithm generates a table of size $T\times N$ where $T$ is the length of the observation sequence. Then, $P(O\mid\lambda)$ is essentially the sum of the last row of this table. The Backward Algorithm works similarly. You may choose one to implement.</div>

<div class="alert alert-block alert-info"><b>Note:</b> You must use dynamic programming in your implementation. Otherwise, the tests will timeout.</div>

Implement your algorithm in the `find_model(O, models)` function below. Do NOT change the function signature. You are, however, free to define and use helper functions.

In [37]:
def model_probability_manual(O, model):

    import numpy as np
    
    A = np.array(model['A'])
    B = np.array(model['B'])
    pi = np.array(model['pi'])
    
    N = len(pi) 
    T = len(O)  
    
    forward_prob = np.zeros((N, T))
    forward_prob[:, 0] = pi * B[:, O[0]]
    
    for t in range(1, T):
        for j in range(N):
            forward_prob[j, t] = np.dot(forward_prob[:, t-1], A[:, j]) * B[j, O[t]]
    
 
    P_O_given_model = np.sum(forward_prob[:, -1])
    log_prob = np.log(P_O_given_model)
    
    return log_prob

def find_model(O, models):

    best_model = 0
    max_prob = -float('inf')
    
    for i, model in enumerate(models):
        prob = model_probability_manual(O, model)
        if prob > max_prob:
            max_prob = prob
            best_model = i
    
    return best_model




### Run & Test
Use the cell below to run and test `find_model(O, models)`. 

In [38]:
if __name__ == '__main__':
    def test_metadata(N, M, T):
        """Gathers constants that describes a test
        Args:
          N: Number of hidden states.
          M: Number of observable states.
          T: Length of observable sequence.
        """
        return {'N': N, 'M': M, 'T': T}
    tests = [test_metadata(N=5, M=3, T=100), 
             test_metadata(N=10, M=5, T=200),
             test_metadata(N=10, M=3, T=300)]    
    output = [['Test', '% of Correct Trials', 'Grade']]
    for idx, test in enumerate(tests):
        print("Running test " + str(idx) + "....")
        count = 0
        for trial in range(50):
            # 1. Create Candidate HMMs
            models = []
            pi = np.ones((test['N']))
            pi = pi / pi.sum()
            for i in range(3):
                models.append({'A': generate_matrix(test['N'], test['N'], i), 
                               'B': generate_matrix(test['N'], test['M'], i), 
                               'pi': pi})

            # 2. Pick 1 HMM
            TRUE_MODEL = random.randint(0,2) 

            # 3. Create observation based on model
            OBS = []
            for i in range(test['T']):
                cur_h_state = 0
                if(i == 0):
                    cur_h_state = np.random.choice(np.arange(0, test['N']), p=models[TRUE_MODEL]['pi'])
                else:
                    cur_h_state = np.random.choice(np.arange(0, test['N']), p=models[TRUE_MODEL]['A'][int(cur_h_state)])
                OBS.append(int(np.random.choice(np.arange(0, test['M']), p=models[TRUE_MODEL]['B'][int(cur_h_state)])))

            # 3. Test find_model
            est_model = find_model(OBS, models)
            if(est_model == TRUE_MODEL):
                count = count + 1
        score = 10 if(count > (50 / len(tests))) else 0
        output.append([idx, '{:.1%}'.format(count / 50), "{:0.0f} / 10".format(score)])  
    output.append(['<i>👻 Hidden test 0 👻</i>','<i>???</i>', '<i>???</i> / 10'])
    output.append(['<i>👻 Hidden test 1 👻</i>','<i>???</i>', '<i>???</i> / 10'])
    display_table(output)

Running test 0....


C:\Users\rohan\AppData\Local\Temp\ipykernel_18536\3478717879.py:21: RuntimeWarning: divide by zero encountered in log
  log_prob = np.log(P_O_given_model)


Running test 1....
Running test 2....


---
## Problem 2. Recovering the Hidden States
You are given a sequence of observed states and an HMM. Recover the most likely sequence of hidden states. You are encouraged to implement the Viterbi algorithm here.

In [39]:
def recover_states(O, model):

    import numpy as np
    
    A = np.array(model['A'])
    B = np.array(model['B'])
    pi = np.array(model['pi'])
    
    N = len(pi)  
    T = len(O)  
    
    V = np.zeros((N, T))  
    P = np.zeros((N, T), dtype=int)  
    
    V[:, 0] = pi * B[:, O[0]]
    
    for t in range(1, T):
        for j in range(N):
            seq_probs = V[:, t-1] * A[:, j]
            P[j, t] = np.argmax(seq_probs)
            V[j, t] = np.max(seq_probs) * B[j, O[t]]
    
    best_last_state = np.argmax(V[:, -1])
    
    S = np.zeros(T, dtype=int)
    S[-1] = best_last_state
    for t in range(T-2, -1, -1):
        S[t] = P[S[t+1], t+1]
    
    return S.tolist()




### Run & Test
Use the cell below to run and test `recover_states(O, model)`. 

In [40]:
if __name__ == '__main__':
    def test_metadata(N, M, T):
        """
        Args:
          N: Number of hidden states.
          M: Number of observable states.
          T: Length of observable sequence.
        """
        return {'N': N, 'M': M, 'T': T}
    def random_guessing(T, M):
        return [random.randint(0,M-1) for i in range(T)]
    tests = [test_metadata(N=5, M=3, T=100), 
             test_metadata(N=10, M=3, T=200),
             test_metadata(N=12, M=3, T=300)]  
    output = [['Test', '% of States Uncovered', 'Random Guessing', 'Grade']]
    for idx, test in enumerate(tests):
        score = 0
        rand_score = 0
        print("Running test " + str(idx) + ".....")
        for trial in range(100):
            # 1. Generate model
            pi = np.ones((test['N']))
            pi = pi / pi.sum()
            model = {'A': generate_matrix(test['N'], test['N'], idx),
                     'B': generate_matrix(test['N'], test['M'], idx),
                     'pi': pi}
            # 2. Generate observations and hidden states
            OBS = []
            TRUE_STATES = []
            for i in range(test['T']):
                cur_h_state = 0
                if(i == 0):
                    cur_h_state = np.random.choice(np.arange(0, test['N']), p=model['pi'])
                else:
                    cur_h_state = np.random.choice(np.arange(0, test['N']), p=model['A'][int(cur_h_state)])
                OBS.append(int(np.random.choice(np.arange(0, test['M']), p=model['B'][int(cur_h_state)])))
                TRUE_STATES.append(int(cur_h_state))
            # 3. Test viterbi
            est_states = recover_states(OBS, model)
            random_states = random_guessing(test['T'], test['M'])
            # 4. Calculate score
            count = sum(x == y for x, y in zip(np.array(TRUE_STATES), np.array(est_states)))
            score = score + count / test['T']
            rand_count = sum(x == y for x, y in zip(np.array(TRUE_STATES), np.array(random_states)))
            rand_score = rand_score + rand_count / test['T']
        grade = 10 if(score > rand_score) else 0
        output.append([idx, '{:.1%}'.format(score / 100), '{:.1%}'.format(rand_score / 100), "{:0.0f} / 10".format(grade)])
    output.append(['<i>👻 Hidden test 0 👻</i>','<i>???</i>', '<i>???</i>', '<i>???</i> / 10'])
    output.append(['<i>👻 Hidden test 1 👻</i>','<i>???</i>', '<i>???</i>', '<i>???</i> / 10'])
    display_table(output)

Running test 0.....
Running test 1.....
Running test 2.....


---
## Rubric
Since HMM are probabilistic models, we will run your code many times and your grade should stabilize after many trials by the Law of Large Numbers. The script provided runs 50 trials to make testing more pleasant. The actual grading script will run more trials. **On Gradescope, we will display your total score, including hidden test cases. You can expect your final grade for this MP to match what you see after submission.**

#### Determing the Best Model (50 points) 
You will be graded on the 3 sets of provided cases as well as 2 sets of hidden cases. 10 points will be rewarded if your algorithm performs better than random guessing. For example, if the test provides 5 models, your algorithm will receive full points if it outputs the correct model over 20% of the times.

#### Recovering the Hidden States (50 points) 
You will be graded on the 3 sets of provided cases as well as 2 sets of hidden cases. 10 points will be rewarded if your algorithm recovered more states than random guessing.

---
## Submission Guideline
This Jupyter notebook is the only file you need to submit on Gradescope.

**Make sure any code you added to this notebook, except for import statements, is either in a function or guarded by `__main__`(which won't be run by the autograder). Gradescope will give you immediate feedback using the provided test cases. It is your responsibility to check the output before the deadline to ensure your submission runs with the autograder.**